## Summary

Simple service which reads images from google and use openai 4o vision to summarize the content.


# Iniiat Setup

OpenAI API KEY
1. goto https://platform.openai.com/settings/organization/api-keys and generate a key
2. create a file named .env in local file system
3. Update the file with the details below
  OPENAI_API_KEY = <<apikey from open ai>>

**Google Cloud Console**
1. Create / Reuse a project
2. Goto API Lib - https://console.cloud.google.com/apis/librar
3. Search and Enable specific APIs
4. Goto APIs Credentials - https://console.cloud.google.com/apis/credentials
5. Create a Serivice Account  - https://console.cloud.google.com/iam-admin/serviceaccounts
6. Download the service account details as a json file.
7. upload to colab and place it at the root folder, name it gcp-key.json



In [ ]:
!pip install openai
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
!pip install python-dotenv

## Import and Load Env valirables

In [ ]:
import os
import dotenv

from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()



## Initilize OpenAI and Google

In [ ]:
# OpenAI
openAIKey = os.environ["OPENAI_API_KEY"]

#Google
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
SERVICE_ACCOUNT_FILE = '/content/gcp-key.json'  # Update with your JSON key file path


## Print all the files in the shared folder

In [ ]:

# Authenticate using the service account
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Build the Drive API service
service = build('drive', 'v3', credentials=creds)

# The folder ID from your shared folder URL
folder_id = '1jpX0N0sdIejjDAGt4D4y5bLA-RYK3dcO'

# Create a query to list files in the folder
query = f"'{folder_id}' in parents"

# Call the Drive API to list files in the folder, including webViewLink
results = service.files().list(q=query,fields="nextPageToken, files(id, name, webViewLink)").execute()
items = results.get('files', [])

# Print file names and URLs (optional)
if not items:
    print('No files found.')
else:
    print('Files found in the folder:')
    for item in items:
        print(f"File Name: {item['name']}, URL: https://drive.google.com/uc?export=view&id={item['id']}")

## Analyze one file at a time

In [ ]:
# Initialize your OpenAI client
client = OpenAI()

# File to store the responses
output_file = "openai_responses.txt"

# Assuming 'items' is obtained from Section 1
for item in items:
    image_url = f"https://drive.google.com/uc?export=view&id={item['id']}"

    # Build and send the request to OpenAI's API
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "This is a screenshot from Andrej Krapathy youtube channel teaching LLM. Identify important concepts shown in the image. summarize"},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url,
                        },
                    },
                ],
            }
        ],
        max_tokens=300,
    )

    # Append the response to the file
    with open(output_file, "a") as f:  # "a" mode for appending
        f.write(response.choices[0].message.content + "\n")

    print(response.choices[0].message.content) # Optional: Print to console as well

